# Optimizing the model parameters

Now that we have a model and data, it's time to train, validate, and test our model by optimizing its parameters on our data. Training a model is an iterative process; in each iteration (*epoch*), the model makes a guess about the output, calculates the error in its guess (*loss*), collects the derivatives of the error with respect to its parameters (as we saw in the previous module), and **optimizes** these parameters using gradient descent. 

## Prerequisite code 

We'll load the code from the previous modules on **Datasets & DataLoaders** and **Build Model**.

In [ ]:
%matplotlib inline
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

## Setting hyperparameters 

Hyperparameters are adjustable parameters that let you control the model optimization process. 
Different hyperparameter values can impact model training and accuracy level.

We define the following hyperparameters for training:
 - **Number of Epochs**: The number times the entire training dataset is passed through the network. 
 - **Batch Size**: The number of data samples the model sees in each epoch. This iterates over the number of batches needed to complete an epoch.
 - **Learning Rate**: The size of steps that the model matches as it searches for the best weights that will produce a higher model accuracy. Smaller values mean the model takes longer to find the best weights. Larger values might result in the model stepping over and missing the best weights, which yields unpredictable behavior during training.

In [3]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## Add an optimization loop

Once we set our hyperparameters, we can train and optimize our model with an optimization loop. Each iteration of the optimization loop is called an **epoch**. 

Each epoch consists of two main parts:
 - **The Train Loop**: Iterates over the training dataset and tries to converge to optimal parameters.
 - **The Validation/Test Loop**: Iterates over the test dataset to check if model performance is improving.

Let's examine some of the concepts used in the training loop. Jump ahead to see the full implementation of the optimization loop.

### Add a loss function

When presented with some training data, our untrained network isn't likely to give the correct answer. A **loss function** measures the degree of dissimilarity of an obtained result to the target value, and it's the loss function that we want to minimize during training. To calculate the loss, we make a prediction using the inputs of our given data sample and compare it against the true data label value.

Common loss functions include:

- `nn.MSELoss`: Mean Square Error, used for regression tasks.
- `nn.NLLLoss`: Negative Log Likelihood, used for classification.
- `nn.CrossEntropyLoss`: Combines `nn.LogSoftmax` and `nn.NLLLoss`.

We pass our model's output logits to `nn.CrossEntropyLoss`, which normalizes the logits and computes the prediction error.

In [4]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

### Optimization pass

Optimization is the process of adjusting model parameters to reduce model error in each training step. **Optimization algorithms** define how this process is performed (in this example we use *Stochastic Gradient Descent*).

All optimization logic is encapsulated in  the `optimizer` object. Here, we use the SGD optimizer; there are many different optimizers available in PyTorch, such as `ADAM` and `RMSProp`, that are suited for different kinds of models and data.

We initialize the optimizer by registering the model's parameters that need to be trained, and passing in the learning rate hyperparameter.

In [5]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

Inside the training loop, optimization happens in three steps:
 * Call `optimizer.zero_grad()` to reset the gradients of model parameters. Gradients by default add up; to prevent double-counting, we explicitly zero them at each iteration.
 * Back-propagate the prediction loss with a call to `loss.backwards()`. PyTorch deposits the gradients of the loss with respect to each parameter. 
 * Once we have our gradients, we call `optimizer.step()` to adjust the parameters by the gradients collected in the backward pass.

## Full implementation

We define a `train_loop` function that loops over our optimization code, and a `test_loop` function that 
evaluates the model's performance against our test data.

In [6]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):        
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

We initialize the loss function and optimizer, and pass it to `train_loop` and `test_loop`.
Feel free to increase the number of epochs to track the model's improving performance.

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

You might have noticed that the model is initially not very good (that's OK). Try running the loop for more `epochs` or adjusting the `learning_rate` to a bigger number. It might also be the case that the model configuration we chose might not be the optimal one for this kind of problem (it isn't). Later courses will delve more into the model shapes that work for vision problems.


## Saving Models

When you're satisfied with the model's performance, you can use `torch.save` to save it. PyTorch models store the learned parameters in an internal state dictionary called `state_dict`. You can persist these with the `torch.save` method:

In [ ]:
torch.save(model.state_dict(), "data/model.pth")

print("Saved PyTorch Model State to model.pth")